# Working with the en_IN locale: numbers

## Setup

In [63]:
!locale -a

C
C.utf8
de_AT.utf8
de_BE.utf8
de_CH.utf8
de_DE.utf8
de_IT.utf8
de_LI.utf8
de_LU.utf8
en_AG
en_AG.utf8
en_AU.utf8
en_BW.utf8
en_CA.utf8
en_DK.utf8
en_GB.utf8
en_HK.utf8
en_IE.utf8
en_IL
en_IL.utf8
en_IN
en_IN.utf8
en_NG
en_NG.utf8
en_NZ.utf8
en_PH.utf8
en_SG.utf8
en_US.utf8
en_ZA.utf8
en_ZM
en_ZM.utf8
en_ZW.utf8
fr_BE.utf8
fr_CA.utf8
fr_CH.utf8
fr_FR.utf8
fr_LU.utf8
POSIX
sv_FI.utf8
sv_SE.utf8
vi_VN
vi_VN.utf8


In [64]:
import locale
loc = "en_IN.UTF-8"

try:
    from icu import Locale, NumberFormatter, CurrencyUnit, Precision, Notation
except:
    !pip install pyicu
    from icu import Locale, NumberFormatter, CurrencyUnit, Precision, Notation

try:
    locale.setlocale(locale.LC_ALL, loc)
except locale.Error:
    print("After the missing locale is installed, the runtime environment will restart, please rerun all code.")
    !sudo locale-gen {loc}
    !sudo update-locale
    import os
    os.kill(os.getpid(), 9)

## Introduction

We will look at how to handle locale sensitive number formatting using the [locale](https://docs.python.org/3/library/locale.html) and _PyICU_ modules, using the `en_IN` locale as an example.

Numeric opeerators available in the _locale_ module

    locale.format_string(format, val, grouping=False, monetary=False)
    locale.format(format, val, grouping=False, monetary=False)
    locale.currency(val, symbol=True, grouping=False, international=False)
    locale.str(float)
    locale.delocalize(string)
    locale.localize(string, grouping=False, monetary=False)
    locale.atof(string, func=float)
    locale.atoi(string)
    
PyICU provides a wrapper around [icu::NuberFormatter](https://unicode-org.github.io/icu-docs/apidoc/released/icu4c/numberformatter_8h.html#details) (icu4c)

`NumberFormatter` supports:

* Decimal Formatting
* Currencies
* Measurement Units
* Percentages
* Scientific Notation
* Compact Notation
    


## Currency

### Using the locale module

`locale.currency(value, symbol=True, grouping=False, international=False)`

__symbol__: Boolean paramter that includes or excludes the current locales' currency symbol from string. \
__grouping__: Applies or suppresses use of grouping seperators in resultant string. Grouping pattern and grouping seperator are locale sensitive. \
__International__: Controls whether the international currency symbol is used is sued instead of the locale's currency symbol.

For the `en_IN.UTF-8` locale, the _Indian rupee sign_ (₹) currency symbol is used. The comma is used as the grouping seperator, a period is used as the decimal seperator and a (2, 3) grouping pattern is used for the digits.

In [65]:
lst = [2.54, 4.0, 3, 9.95, 5.4]
print([locale.currency(i, grouping=True) for i in lst])

['₹ 2.54', '₹ 4.00', '₹ 3.00', '₹ 9.95', '₹ 5.40']


In [66]:
lst2 = [123452.54, 14367843.0, 120030, 900020.95, 5000000.0]
print([locale.currency(i, grouping=True) for i in lst2])


['₹ 1,23,452.54', '₹ 1,43,67,843.00', '₹ 1,20,030.00', '₹ 9,00,020.95', '₹ 50,00,000.00']


Depending on the `en_IN.UTF-8` locale definition on the OS being used

In [67]:
locale._override_localeconv["p_sep_by_space"] = 1
print([locale.currency(i, grouping=True) for i in lst2])


['₹ 1,23,452.54', '₹ 1,43,67,843.00', '₹ 1,20,030.00', '₹ 9,00,020.95', '₹ 50,00,000.00']


### Using PyICU


In [68]:
icu_loc = Locale.createCanonical(loc)
formatter = NumberFormatter.with_().unit(CurrencyUnit('INR')).locale(icu_loc)
# print([str(formatter.formatDouble(i)).replace('\xa0', ' ') for i in lst2])
print([str(formatter.formatDouble(i)) for i in lst2])

['₹1,23,452.54', '₹1,43,67,843.00', '₹1,20,030.00', '₹9,00,020.95', '₹50,00,000.00']


In [69]:
formatter2 = NumberFormatter.withLocale(icu_loc).unit(CurrencyUnit('INR'))
print([str(formatter2.formatDouble(i)) for i in lst2])

['₹1,23,452.54', '₹1,43,67,843.00', '₹1,20,030.00', '₹9,00,020.95', '₹50,00,000.00']


In [84]:
formatter3 = NumberFormatter.with_().unit(CurrencyUnit('INR')).locale(icu_loc).precision(Precision.minMaxFraction(2,2))
print([formatter3.formatDouble(i) for i in lst2])

['₹1,23,452.54', '₹1,43,67,843.00', '₹1,20,030.00', '₹9,00,020.95', '₹50,00,000.00']


In [85]:
formatter4 = NumberFormatter.with_().unit(CurrencyUnit('INR')).locale(icu_loc).precision(Precision.minMaxFraction(2,2)).notation(Notation.compactShort())
print([formatter4.formatDouble(i) for i in lst2])

['₹1.23L', '₹1.44Cr', '₹1.20L', '₹9.00L', '₹50.00L']


In [90]:
formatter5 = NumberFormatter.with_() \
    .unit(CurrencyUnit('INR')) \
    .locale(icu_loc) \
    .precision(Precision.minMaxFraction(2,2)) \
    .notation(Notation.compactShort())
print([formatter5.formatDouble(i) for i in lst2])

['₹1.23L', '₹1.44Cr', '₹1.20L', '₹9.00L', '₹50.00L']


---

In [88]:
import pdir
pdir(formatter3)

special attribute:
    __class__, __doc__
abstract class:
    __subclasshook__
object customization:
    __format__, __hash__, __init__, __new__, __repr__, __sizeof__, __str__
rich comparison:
    __eq__, __ge__, __gt__, __le__, __lt__, __ne__
attribute access:
    __delattr__, __dir__, __getattribute__, __setattr__
class customization:
    __init_subclass__
pickle:
    __reduce__, __reduce_ex__
descriptor:
    owned: class getset_descriptor with getter, setter, deleter
function:
    decimal: 
    formatDecimal: 
    formatDecimalToValue: 
    formatDouble: 
    formatDoubleToValue: 
    formatInt: 
    formatIntToValue: 
    grouping: 
    integerWidth: 
    notation: 
    perUnit: 
    precision: 
    roundingMode: 
    scale: 
    sign: 
    symbols: 
    toSkeleton: 
    unit: 
    unitWidth: 
    usage: 

In [73]:
pdir(Precision)

special attribute:
    __class__, __doc__
abstract class:
    __subclasshook__
object customization:
    __format__, __hash__, __init__, __new__, __repr__, __sizeof__, __str__
rich comparison:
    __eq__, __ge__, __gt__, __le__, __lt__, __ne__
attribute access:
    __delattr__, __dir__, __getattribute__, __setattr__
class customization:
    __init_subclass__
pickle:
    __reduce__, __reduce_ex__
descriptor:
    currency: class classmethod_descriptor with getter
    fixedFraction: class classmethod_descriptor with getter
    fixedSignificantDigits: class classmethod_descriptor with getter
    increment: class classmethod_descriptor with getter
    integer: class classmethod_descriptor with getter
    maxFraction: class classmethod_descriptor with getter
    maxSignificantDigits: class classmethod_descriptor with getter
    minFraction: class classmethod_descriptor with getter
    minMaxFraction: class classmethod_descriptor with getter
    minMaxSignificantDigits: class classmethod_desc

In [76]:
pdir(CurrencyUnit)

special attribute:
    __class__, __doc__
abstract class:
    __subclasshook__
arithmetic:
    __mul__, __pow__, __rmul__, __rpow__, __rtruediv__, __truediv__
object customization:
    __format__, __hash__, __init__, __new__, __repr__, __sizeof__, __str__
rich comparison:
    __eq__, __ge__, __gt__, __le__, __lt__, __ne__
attribute access:
    __delattr__, __dir__, __getattribute__, __setattr__
class customization:
    __init_subclass__
pickle:
    __reduce__, __reduce_ex__
descriptor:
    createAcre: class classmethod_descriptor with getter
    createAcreFoot: class classmethod_descriptor with getter
    createAmpere: class classmethod_descriptor with getter
    createArcMinute: class classmethod_descriptor with getter
    createArcSecond: class classmethod_descriptor with getter
    createAstronomicalUnit: class classmethod_descriptor with getter
    createAtmosphere: class classmethod_descriptor with getter
    createBar: class classmethod_descriptor with getter
    createBarrel: cl